In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('20250624_skn14_llm')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [5]:
# 함수 리팩토링

def anki_vocab(word, model='gpt-4o-mini', temperature=1, top_p = 1):

    # 역할/페르소나/지시사항
    system_message = """
    1. **시스템 지시사항**

   * LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
   * 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
   * 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
   * 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

2. **예시**

   * 입력 텍스트 예시:

     ```
     The committee will convene next week to discuss the new policy proposals and reach a consensus.
     ```
   * 출력 예시:

     ```
     convene;to come together for a meeting;verb;The committee will convene next week.
     consensus;general agreement;noun;The group reached a consensus after a long discussion.
     proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
     ```

3. **LLM 출력 형식**

   * 각 카드: `단어 또는 어구;뜻;품사;영어 예문`
   * 모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
   * 불필요한 부가설명, 줄바꿈, 문장 없음
    """

    user_message = f"""
    다음 단어를 단어장에 입력해주세요
    {word}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response.choices[0].message.content

word = 'run', 'happy', 'rainy'
output = anki_vocab(word)
print(output)

```
run;to move swiftly on foot;verb;He loves to run every morning.
happy;feeling or showing pleasure or contentment;adjective;She was happy to receive the good news.
rainy;characterized by rain;adjective;It was a rainy day, perfect for staying indoors.
```


In [9]:
import pandas as pd

entries = [line.strip().split(";") for line in output.strip().split("\n")]

df = pd.DataFrame(entries[1:-1], columns=["Word/Phrase", "Definition", "PoS", "Example Sentence"])
df

,Word/Phrase,Definition,PoS,Example Sentence
0,run,to move swiftly on foot,verb,He loves to run every morning.
1,happy,feeling or showing pleasure or contentment,adjective,She was happy to receive the good news.
2,rainy,characterized by rain,adjective,"It was a rainy day, perfect for staying indoors."
